In [1]:
# Import dependencies (python libraries)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# add city libraries
from citipy import citipy

# add request modules
import requests

# import the API key from config.py file
from config import weather_api_key

# import datetime to recast api date a datetime format
import time
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
# Note: lats ranges from -90 to 90
# Note: lngs range -180 t 180

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-90.000, high=90.000, size=2000)

# the zip method combines the lats and lngs into a list of pairs
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)


In [3]:
# Generate Random World Cities
# This requires citypy to be installed to the PythonData enviroment

# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

#cities

746

In [4]:
# Start URL for Weather API Call.
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

#print(url)

In [5]:
# Make a request for Data to an API
# Get the city Weather Data

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
       
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        current_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": current_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 2 of Set 1 | vieux-habitants
Processing Record 3 of Set 1 | san antonio
Processing Record 4 of Set 1 | ushuaia
Processing Record 5 of Set 1 | kavaratti
Processing Record 6 of Set 1 | victoria
Processing Record 7 of Set 1 | jamestown
Processing Record 8 of Set 1 | heiligenhafen
Processing Record 9 of Set 1 | hermanus
Processing Record 10 of Set 1 | goderich
Processing Record 11 of Set 1 | tasiilaq
Processing Record 12 of Set 1 | punta arenas
Processing Record 13 of Set 1 | srivardhan
Processing Record 14 of Set 1 | desaguadero
Processing Record 15 of Set 1 | margate
Processing Record 16 of Set 1 | taltal
Processing Record 17 of Set 1 | hambantota
Processing Record 18 of Set 1 | rio grande
Processing Record 19 of Set 1 | cidreira
Processing Record 20 of Set 1 | port elizabeth
Processing Record 21 of Set 1 | cape town
Processing Record 22 of 

In [6]:
len(city_data)

691

In [12]:
# Create  a DataFrame of City Weather Data
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Vieux-Habitants,16.0589,-61.7659,84.70,69,18,12.84,GP,few clouds
1,San Antonio,29.4241,-98.4936,103.06,37,40,14.97,US,scattered clouds
2,Ushuaia,-54.8000,-68.3000,35.85,92,100,5.39,AR,overcast clouds
3,Kavaratti,10.5669,72.6420,80.94,80,100,22.03,IN,overcast clouds
4,Victoria,22.2855,114.1577,87.33,80,100,8.01,HK,overcast clouds
5,Jamestown,42.0970,-79.2353,87.06,52,0,14.97,US,clear sky
6,Heiligenhafen,54.3667,10.9833,73.74,51,1,9.78,DE,clear sky
7,Hermanus,-34.4187,19.2345,44.80,73,33,4.72,ZA,scattered clouds
8,Goderich,43.7501,-81.7165,78.91,81,87,4.52,CA,overcast clouds
9,Tasiilaq,65.6145,-37.6368,40.44,80,41,1.01,GL,scattered clouds


In [15]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Vieux-Habitants,GP,16.0589,-61.7659,84.70,69,18,12.84,few clouds
1,San Antonio,US,29.4241,-98.4936,103.06,37,40,14.97,scattered clouds
2,Ushuaia,AR,-54.8000,-68.3000,35.85,92,100,5.39,overcast clouds
3,Kavaratti,IN,10.5669,72.6420,80.94,80,100,22.03,overcast clouds
4,Victoria,HK,22.2855,114.1577,87.33,80,100,8.01,overcast clouds
5,Jamestown,US,42.0970,-79.2353,87.06,52,0,14.97,clear sky
6,Heiligenhafen,DE,54.3667,10.9833,73.74,51,1,9.78,clear sky
7,Hermanus,ZA,-34.4187,19.2345,44.80,73,33,4.72,scattered clouds
8,Goderich,CA,43.7501,-81.7165,78.91,81,87,4.52,overcast clouds
9,Tasiilaq,GL,65.6145,-37.6368,40.44,80,41,1.01,scattered clouds


In [16]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")